import required package

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import requests
from bs4 import BeautifulSoup
from bs4 import NavigableString
import pandas as pd
import re


setup selemium 

In [2]:
# Set up selenium to use chrome
options = Options()
#options.add_argument("--headless")
 #No need to open a browser window
driver = webdriver.Chrome(r'C:\Users\admin\Desktop\hw\econ4130\horse\chromedriver', options = options)

# Example of manaully specifying the WebDriver's location: 
# driver = webdriver.Firefox(executable_path="../Others/geckodriver.exe",options=options) #Windows
# driver = webdriver.Firefox(executable_path="../Others/geckodriver",options=options) #Linux


this function is to get the sibling tag

In [3]:
def get_sibling(tag,previous=False):
    if previous:
        sibling = tag.previous_sibling
        while isinstance(sibling, NavigableString):
            sibling = sibling.previous_sibling
    else:
        sibling = tag.next_sibling
        while isinstance(sibling, NavigableString):
            sibling = sibling.next_sibling        
    return sibling

This function is to scrape the place, gear and comment

In [4]:
def scrape_detail(url):
        # Function to access a page and save all horses into a list

    # Fetch the page
    driver.get(url)
    
    # Is there anything?
    if driver.page_source.find("No information.") != -1:
        return []
    
    # Wait 30 secs so that the dynamic content has time to load.
    # Proceed to next date if page doesn't load.
 
    
    # Load the page into BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # Find all tags with href containing "horseno"
    detail = soup.find_all("table",class_="table_bd")
    detail[0].get_text()
    re.findall('\\n(B)', str(detail[0].text.strip()))
    i=0
    table = soup.findChildren('table', class_="table_bd")[0]
    place = re.findall(r'<td style=\"text-align: center;\">(\d+)</td>\n<', str(table))
    gear = re.findall(r'\"width:36px; text-align: center;\">(.+)<', str(table))
    comment = [re.findall(r'<td class=\"f_fs16 f_tal\">\s+([A-z].+)', str(soup.find_all("td",class_="f_fs16")[i]))[0] for i in range(0, len(soup.find_all("td",class_="f_fs16")))]
    lis = [[place[i], gear[i], comment[i]] for i in range(0, len(place))]
    detail_df = pd.DataFrame.from_records(lis, columns = ['Placing', 'Gear', 'Comment'])
    
    return detail_df


this function is to scrape the sector time

In [5]:
def scrape_sec_time(url):
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    stime1 = re.findall(r'<p>\d{2}\.\d{2}|/racing/content/images/StaticFile/blank_img.gif', str(soup))
    stime2 = [''.join(re.findall(r'\d{2}\.\d{2}|/racing/content/images/StaticFile/blank_img.gif', stime1[i])) for i in range(0, len(stime1))]
    stime3 = [''.join(re.sub(r'/racing/content/images/StaticFile/blank_img.gif', 'NA', stime2[i])) for i in range(0, len(stime1))]
    time_list = [stime3[x:x+6] for x in range(0, len(stime3),6)]
    time_df = pd.DataFrame.from_records(time_list, columns = ['1st Sec.', '2st Sec.', '3st Sec.', '4st Sec.', '5st Sec.', '6st Sec.'])
    return time_df


this function is to scrape the table from the hkjc website

In [6]:
def scrape_horses(url):
    # Function to access a page and save all horses into a list

    # Fetch the page
    driver.get(url)
    
    # Is there anything?
    if driver.page_source.find("No information.") != -1:
        return []
    
    # Wait 30 secs so that the dynamic content has time to load.
    # Proceed to next date if page doesn't load.
    try:
        wait = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CLASS_NAME, "f_fs13")))
    except:
        return []
    
    # Load the page into BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find all tags with href containing "HorseId"
    horses = soup.find_all(href=re.compile("HorseId"))
    
    #find all tags with race number
    race_no_soup = soup.find_all("tr",class_="bg_blue color_w font_wb")[0]
    
    
    #find all tags with race info
    race_info = soup.find_all("tbody",class_="f_fs13")[0]
    
  
    

    # 'output_list' is the whole table
    # 'output' is a single row
    output_list = [['horse no', 'Horse', 'Jockey', 'Trainer', 'Actual Wt.', 'Declar. Horse Wt.', 'Draw', 'LBW', 'Running Position 1st sec', 'Running Position 2st sec', 'Running Position 3st sec', 'Running Position 4st sec', 'Running Position 5st sec', 'Running Position 6st sec' ,'Running Position', 'Finish Time', 'Win Odds', 'race no.','Class', 'length', 'grade', 'Going', 'com_name', 'course', 'prize']]
    count = 0
    # Loop through horses
    for horse in horses:
        
        #race info
        race_no = re.findall(r'RACE\s\d\s\((\d+)', race_no_soup.text.strip())[0]
        race_info_list = race_info.text.strip()
        class_type = re.findall(r'(Class\s\d).{3}(\d+M).{4}(\d+-\d+)', race_info_list)[0][0]
        length = re.findall(r'(Class\s\d).{3}(\d+M).{4}(\d+-\d+)', race_info_list)[0][1]
        grade = re.findall(r'(Class\s\d).{3}(\d+M).{4}(\d+-\d+)', race_info_list)[0][2]
        going = re.findall(r'Going :\n(.*)', race_info_list)[0]
        com_name = re.findall(r'([A-Z].*)\nCourse', race_info_list)[0]
        course = re.findall(r'Course :\n(.*)', race_info_list)[0]
        prize = re.findall(r'HK\$\s(.*)', race_info_list)[0]
        raceinfo = [class_type, length, grade, going, com_name, course, prize]
        # Get the horse name
        output = []        
        
        
        # This while loop fetch all remaining fields in a row
        a = get_sibling(horse.parent, previous = True)
        while a != None:
            output.append(a.text
                          .strip()
                          # The last two lines are for running positions
                          .replace('\n','') 
                          .replace(' '*20,' ') 
                         )
            a = get_sibling(a)
        
        
                          
        #append other info to the output
        [output.append(race_no)]
        [output.append(i) for i in raceinfo]        
        # Append each row to the output list
        output_list.append(output) 
        count += 1
        
    for i in range(1, len(output_list)):
        run_pos = output_list[i][8].split('   ')
        while len(run_pos) < 6:
            run_pos.extend('N')
        output_list[i][8:8] = run_pos
        output_df = pd.DataFrame(output_list[1:len(output_list)], columns = output_list[0])
         

    return output_df

this function is to scrape the odd of place

In [7]:
def scrape_place_odd(url):
    # Fetch the page
    driver.get(url)
    
    # Is there anything?
    if driver.page_source.find("No information.") != -1:
        return []
    
    # Wait 30 secs so that the dynamic content has time to load.
    # Proceed to next date if page doesn't load.
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    dividend = []
    dividend_info = soup.find_all("td",class_="f_fs14")
    for d in dividend_info:             #leave with white space
        dividend.append(d.text.strip())
    win_comb = [dividend[2*i] +'//' + dividend[2*i+1] for i in range(0, int(len(dividend)/2))]    
    place = win_comb[1:4]
    place_list = [i.split('//') for i in place]
    for j in place_list:
        j[1] = float(j[1])/10        
    place_df = pd.DataFrame(place_list, columns = ['horse no', 'place odd'])
    return place_df

this function is to scrape the previous rtg and the previous race number

In [8]:
def scrape_previous_game(url):
    # Function to access a page and save all horses into a list

    # Fetch the page
    driver.get(url)
    
    # Is there anything?
    if driver.page_source.find("No information.") != -1:
        return []
    
    # Wait 30 secs so that the dynamic content has time to load.
    # Proceed to next date if page doesn't load.
    # Load the page into BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    race_no = re.findall(r'(\d{3})<\/a>', str(soup))
    pre_race = race_no[1:len(race_no)]
    pre_race.append('NA')
    #rtg = re.findall(r't">(\d{2})<\/td>\n<td align="center" class="htable_text">', str(soup))
    horse_name = soup.find_all("span",class_="title_text")[0].text.strip()
    horse_name = re.findall(r'(.*)\s(\(.*\d\))', horse_name)
    horse_name = ''.join(horse_name[0])
    combine = [[horse_name, race_no[i], pre_race[i]]  for i in range(0, len(race_no))]
    previous_df = pd.DataFrame(combine, columns = ['Horse', 'current race', 'previous race'])
    return previous_df

this loop is for the function scrape_horses and scrape_place_odd

In [ ]:
#URL of data
url_front = "http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/"
url_front_1 = 'https://racing.hkjc.com/racing/information/english/Reports/CORunning.aspx?Date='
url_front_2 = '&RaceNo='
url_front_3 = r'https://racing.hkjc.com/racing/information/english/Racing/DisplaySectionalTime.aspx?RaceDate='
url_front_4 = r'&RaceNo='
horse_full_df = pd.DataFrame()
#Write a loop to go through year, month and day
#Note that month and day is always 2 digit
#Call scrape_horses() in each iteration
for year in range(2015,2021):
    for month in range(1,13):
        for day in range(1,32):
            for page in range(1,12):
                try:
                                        #Convert month and day to 2-digit representation
                    month_2d = '{:02d}'.format(month)
                    day_2d = '{:02d}'.format(day)
                    page_1d = '{:01d}'.format(page)
                    url = url_front + str(year) + month_2d + day_2d + page_1d
                    url1 = url_front_1 + str(year) + month_2d + day_2d + url_front_2 + page_1d
                    url2 = url_front_3  + day_2d + '/' + month_2d + '/' + str(year)+ url_front_4 + page_1d                
                    horse_df = scrape_horses(url)                
                    place_odd_df = scrape_place_odd(url)
                    detail_df = scrape_detail(url1)
                    sec_time_df = scrape_sec_time(url2)
                    #concat the dataframe
                    horse_table_df =  pd.concat([horse_df, place_odd_df, sec_time_df, detail_df], axis = 1)
                    horse_full_df = horse_full_df.append(horse_table_df)
                    print(url)           
                except:
                    pass

http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201501011
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201501012
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201501013
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201501014
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201501015
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201501016
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201501017
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201501018
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201501041
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201501042
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201501043
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201501044
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201501045
http://racing.hkjc.com/ra

http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201502217
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201502219
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201502251
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201502252
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201502253
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201502254
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201502255
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201502256
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201502257
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201502259
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201503011
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201503012
http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/201503013
http://racing.hkjc.com/ra

In [ ]:
horse_full_df


In [ ]:
import csv
horse_full_df.to_csv(r'C:\Users\admin\Desktop\hw\econ4130\horse_full_df.csv')

In [ ]:
horse_blood_df = pd.read_csv(r'C:\Users\admin\Desktop\hw\econ4130\horse\horse_info.csv')
horse_name_list = horse_blood_df['Horse_name']
horse_code = [''.join(re.findall(r'([A-Z]\d{3})', name)) for name in horse_name_list]

In [ ]:
url_front_5 = "https://racing.hkjc.com/racing/information/english/Horse/Horse.aspx?HorseNo="
horse_previos_df = pd.DataFrame()
for code in horse_code:
    try:
        url4 = url_front_5 + code
        previous_df = scrape_previous_game(url4)
        horse_previos_df = horse_previos_df.append(previous_df)
        print(url4)
    except:
        pass

In [ ]:
horse_previos_df.to_csv(r'C:\Users\admin\Desktop\hw\econ4130\horse_previous_df.csv')

data cleaning and feature engineering

In [ ]:
import pandas as pd
import numpy as np
import re
#merge column
horse_table_df = pd.read_csv(r'C:\Users\admin\Desktop\hw\econ4130\horse_full_df.csv')
horse_previous_df = pd.read_csv(r'C:\Users\admin\Desktop\hw\econ4130\horse_previous_df.csv')
horse_info = pd.read_csv(r'C:\Users\admin\Desktop\hw\econ4130\horse\horse_info.csv')
horse_table_df = horse_table_df.iloc[:,1:len(horse_table_df.columns)]
horse_previous_df.rename(columns={'current race':'race no.'}, inplace = True)
horse_previous_df = horse_previous_df.iloc[:,1:len(horse_previous_df.columns)]

table_prerace = pd.merge(horse_table_df, horse_previous_df, on=['race no.', 'Horse'], how='inner')

#identify the 'curent' data
new = table_prerace[['horse no', 'Horse', 'Jockey', 'Trainer', 'Class', 'Actual Wt.', 'Declar. Horse Wt.', 'grade', 'Going', 'com_name', 'Gear','course','previous race', 'Win Odds', 'place odd', 'race no.', 'Placing']]

for col in new.columns: 
    new.rename(columns={col:('new_' + col)}, inplace = True)

#rename previous race  
table_prerace.rename(columns={'race no.':'previous race', 'previous race':'new_previous race'}, inplace = True)
new.rename(columns={'new_previous race':'previous race', 'new_Horse':'Horse'}, inplace = True)

#merge the new data and the previous data
new_old = pd.merge(new, table_prerace, on=['previous race', 'Horse'], how='inner')

import csv
#new_old.to_csv(r'C:\Users\admin\Desktop\hw\econ4130\new_old.csv')
#
new_old['new_Declar. Horse Wt.'] = pd.to_numeric(new_old['new_Declar. Horse Wt.'], errors='coerce')
new_old['Declar. Horse Wt.'] = pd.to_numeric(new_old['Declar. Horse Wt.'], errors='coerce')

#feature engineering
#find the weight and actual weight difference
weight_diff = new_old['new_Actual Wt.'] - new_old['Actual Wt.']
declare_wt_diff = new_old['new_Declar. Horse Wt.'] - new_old['Declar. Horse Wt.']
new_old['weight_diff'] = weight_diff
new_old['declare_wt_diff'] = declare_wt_diff
#merge horse blood
#change the horse name
def change_name(name):
    return ''.join(re.findall(r'(.+)\s(\(.+\))', name)[0])
horse_info['Horse_name'] = horse_info['Horse_name'].apply(change_name)
horse_info.rename(columns={'Horse_name':'Horse'}, inplace = True)

#
new_old['Horse']
horse_df = pd.merge(horse_info, new_old, on = ['Horse', 'Trainer'], how='inner')
#clean all --- to nan
horse_df = horse_df.replace('---', '')
horse_df = horse_df.replace('N', '')
horse_df.to_csv(r'C:\Users\admin\Desktop\hw\econ4130\horse_df.csv')


import require package

In [1]:
import os
os.chdir(r'C:\Users\admin\Desktop\hw\econ4130\Completed')
import warnings
warnings.filterwarnings("ignore")
import gc
# load libraries
import numpy
from sklearn import datasets
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import matplotlib
matplotlib.use('Agg')
from bayes_opt import BayesianOptimization
from matplotlib import pyplot
pyplot.style.use('ggplot')          
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from  sklearn.datasets  import  make_hastie_10_2
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss, matthews_corrcoef, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
import contextlib
from sklearn.metrics import plot_confusion_matrix
import seaborn as sn
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn import utils
from sklearn.tree import export_graphviz
from sklearn.tree import plot_tree

read data and one hot encoding

In [5]:
horse_df = pd.read_csv(r'C:\Users\admin\Desktop\hw\econ4130\horse_df.csv')
horse_df = horse_df.iloc[:,1:len(horse_df.columns)]
a = horse_df.dtypes

refer_index = []
horse_df['new_Placing'] = horse_df['new_Placing'].fillna(0)
c = horse_df['new_Placing']
np.unique(c)
horse_df['new_Placing'].dtype

#one hot encoding
for col in horse_df.columns:
    try:
        if horse_df.dtypes[col] != 'float64':
            horse_df[col] = pd.factorize(horse_df[col])[0]
            refer_index.append(pd.factorize(horse_df[col])[1])
    except:
        pass
horse_df_sorted = horse_df.sort_values(by=['new_race no.'])
horse_df_sorted.describe()
cols = list(horse_df_sorted.columns)
a , b = cols.index('new_Placing'), cols.index('declare_wt_diff')
cols[b], cols[a] = cols[a], cols[b]
horse_df_sorted = horse_df_sorted[cols]
horse_wp_sorted = horse_df_sorted
#change here
#horse_df_sorted = horse_df_sorted.drop(columns = ['place odd', 'new_place odd'])
horse_df_sorted = horse_df_sorted.drop(columns = ['place odd', 'new_place odd', 'new_Win Odds'])
X, y = horse_df_sorted.iloc[:,:-1], horse_df_sorted.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle = False)
# horse with place odd
wpx , wpy = horse_wp_sorted.iloc[:,:-1], horse_wp_sorted.iloc[:,-1]
no1, y1, n2, y2 = train_test_split(wpx, wpy, test_size=0.25, shuffle = False)
data_dmatrix_train = xgb.DMatrix(data=X_train,label=y_train)
data_dmatrix_test =  xgb.DMatrix(data=X_test,label=y_test)

construct a decision tree for demo

In [7]:
#decision tree
no_na = no1.fillna(0)
n2_cat = n2.astype('category')
lab_enc = preprocessing.LabelEncoder()
model=DecisionTreeClassifier(max_depth=2,criterion='entropy')
model.fit(no_na,n2_cat)
plt.figure(figsize=(15,10))
a = plot_tree(model, 
              feature_names=list(no_na.columns), 
              class_names=str(n2_cat.unique()), 
             filled=True, 
              rounded=True, 
              fontsize=8)
pyplot.savefig('decision.png')

baysian optimization
first define a function

In [8]:
early_stopping = 10
nc = len(np.unique(y))
def xgb_evaluate(max_depth, gamma, colsample_bytree, n_estimators, eta, subsample):
    params = {'objective': 'multi:softmax',
              'eval_metric': 'mlogloss',
              'max_depth': int(max_depth),
              'subsample': subsample,
              'eta': eta,
              'gamma': gamma,
              'nthread':8,
              'num_class':nc,
              'n_estimators':int(n_estimators),
              'colsample_bytree': colsample_bytree}
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, data_dmatrix_train, nfold=10)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -1.0 * cv_result['test-mlogloss-mean'].iloc[-1]

set the range want to search and search the optimal point

In [9]:
xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (0, 10), 
                                         'gamma': (0, 5),
                                         'colsample_bytree': (0, 1),
                                         'subsample': (0, 1),
                                         'eta':(0.0001, 1),
                                         'n_estimators': (5,1000),
                                         })
# Use the expected improvement acquisition function to handle negative numbers
# Optimally needs quite a few more initiation points and number of iterations
xgb_bo.maximize(init_points=10, n_iter=5, acq='ei')
target = [xgb_bo.res[i]['target'] for i in range(0, len(xgb_bo.res))]
for i in range(0, len(target)):
    if xgb_bo.res[i]['target'] == max(target):
        best_param = xgb_bo.res[i]

|   iter    |  target   | colsam... |    eta    |   gamma   | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
|  1        | -2.732    |  0.4223   |  0.6477   |  1.649    |  8.357    |  147.6    |  0.7093   |
|  2        | -2.605    |  0.7104   |  0.6182   |  2.991    |  6.053    |  238.1    |  0.7401   |
|  3        | -2.574    |  0.00391  |  0.735    |  4.668    |  7.983    |  726.4    |  0.364    |
|  4        | -5.386    |  0.2346   |  0.94     |  1.705    |  7.418    |  921.6    |  0.1049   |
|  5        | -2.453    |  0.721    |  0.4301   |  0.9619   |  1.647    |  891.6    |  0.6294   |
|  6        | -2.836    |  0.7579   |  0.9319   |  0.5994   |  5.64     |  26.33    |  0.7023   |
|  7        | -2.463    |  0.9349   |  0.2682   |  2.207    |  2.453    |  798.5    |  0.9755   |
|  8        | -3.214    |  0.3132   |  0.6957   |  4.04     |  8.57     |  519.6    |  0.01241  |
|  9        | -2.607

define the parameter

In [10]:
#1st
#params = {'eta':   0.76848176873423635, 'max_depth': 2, 'subsample': 0.9, 'colsample_bytree': 0.733283853800647, 'objective': 'multi:softmax', 'seed': 4130, 'silent': 1, 'eval_metric':'mlogloss', 'nthread':8, 'num_class':nc, 'n_estimators': 246,'gamma': 0.12927266469325016, 'subsample': 0.7223685761352556}

#2st
params = {'eta':   0.4706790997792266, 'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 0.7907512352163658, 'objective': 'multi:softmax', 'seed': 39, 'silent': 1, 'eval_metric':'mlogloss', 'nthread':8, 'num_class':nc, 'n_estimators': 894,'gamma': 4.858986393127731, 'subsample': 0.929637360190657}

loop for the different seed to find the best parameter

In [15]:
result_seed1 = []
for i in range(0, 10):
    
    params = {'eta':   0.4706790997792266, 'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 0.7907512352163658, 'objective': 'multi:softmax', 'seed':i, 'silent': 1, 'eval_metric':'mlogloss', 'nthread':8, 'num_class':nc, 'n_estimators': 894,'gamma': 4.858986393127731, 'subsample': 0.929637360190657}
    xg_clf = xgb.train(params=params, dtrain=data_dmatrix_train)
    preds = xg_clf.predict(data_dmatrix_test)
    y_true= y_test
    result_seed1.append([i, metrics.accuracy_score(y_true, preds)])
    print("Accuracy : %.4g" % metrics.accuracy_score(y_true, preds))
    
be = [result_seed[i][1] for i in range(0, len(result_seed))]
max(be)
np.mean(be)
for i in range(0, len(be)):
    if result_seed1[i][1] == max(be):
        best_seed = result_seed1[i]
#plot xgb_bo.res
param_list = [xgb_bo.res[i]['params'] for i in range(0, len(xgb_bo.res))]

Accuracy : 0.1365
Accuracy : 0.1408
Accuracy : 0.1471
Accuracy : 0.1381
Accuracy : 0.1456
Accuracy : 0.1408
Accuracy : 0.133
Accuracy : 0.1365
Accuracy : 0.1404
Accuracy : 0.1393


plot the relationship between each parameters and the error

In [16]:
target_df = pd.DataFrame(target)
bay_result = pd.DataFrame(param_list)
bay_result = pd.concat([target_df,bay_result], axis = 1)
bay_result = bay_result.sort_values(by=0, ascending=False)
plt.figure(figsize = (20,8))
#plt.plot(bay_result['colsample_bytree'], bay_result[0])
#plt.plot(bay_result['eta'], bay_result[0])
#plt.plot(bay_result[0], bay_result['gamma'])
#plt.plot(bay_result[0], bay_result['max_depth'])
plt.plot(bay_result[0], bay_result['n_estimators'])
#plt.plot(bay_result['subsample'], bay_result[0])
plt.legend(['number of estimators'], loc='upper left', fontsize = 18)
pyplot.ylabel('mlogloss', fontsize=20)
pyplot.show()
pyplot.savefig('bay.png')

grid search

In [ ]:
#cv.head()
#print((cv["test-mlogloss-mean"]).tail(1))
####################################
# grid search
# model = xgb.XGBClassifier()
# n_estimators = [100, 200, 300, 400, 500]
# learning_rate = [0.0001, 0.001, 0.01, 0.1]
    

# param_grid = dict(learning_rate=learning_rate, n_estimators=n_estimators)
# kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
# grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
# grid_result = grid_search.fit(X, y)

# # summarize results
# print(); print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']

# for mean, stdev, param in zip(means, stds, params):
# 	     print("%f (%f) with: %r" % (mean, stdev, param))
#      # plot results
# pyplot.subplots(figsize=(12,12))
# scores = numpy.array(means).reshape(len(learning_rate), len(n_estimators))

# for i, value in enumerate(learning_rate):
#     pyplot.plot(n_estimators, scores[i], label='learning_rate: ' + str(value))

# pyplot.legend()
# pyplot.xlabel('n_estimators')
# pyplot.ylabel('Log Loss')
# pyplot.show()
# pyplot.savefig('n_estimators_vs_learning_rate.png')  

train the model with the best parameter, plot vip and confustion matrix

In [17]:
xg_clf = xgb.train(params=params, dtrain=data_dmatrix_train)
preds = xg_clf.predict(data_dmatrix_test)
y_true= y_test
print("Accuracy : %.4g" % metrics.accuracy_score(y_true, preds))
#
plt.rcParams['figure.figsize'] = [10, 10]
plt.show()

xgb.plot_importance(xg_clf, max_num_features=10)
plt.rcParams['figure.figsize'] = [150,50]
plt.show()
plt.savefig('vip.png')
#confusion matrix
plt.figure(figsize = (10,8))
tab = pd.DataFrame(confusion_matrix(y_true, preds))
tab = tab.drop(0, axis = 1)
tab = tab.drop(0, axis = 0)
sn.heatmap(tab, annot=True, annot_kws={"size": 7})

pyplot.show()
pyplot.savefig('confusion.png')
#accuracy condition on 1
tab[1][1]/(sum(tab[1])+sum(tab[:1]))

Accuracy : 0.1393


back testing for win

In [18]:
odd_df = pd.concat([y1[['new_Win Odds','new_place odd']], y_true], axis = 1)
win_df = odd_df[np.array(y_true) == preds]
win_df = win_df[win_df['new_Placing'] == 1]
#cumsum
lose_df = np.repeat(-1,(len(preds[preds == 1]) - len(win_df['new_Win Odds'])))

combine_df = pd.concat([win_df['new_Win Odds'], pd.DataFrame(lose_df)])
combine_df = combine_df.sample(frac=1).reset_index(drop=True)
cum_win = np.cumsum(combine_df)*10
cum_win.plot()
plt.rcParams['figure.figsize'] = [10,10]
plt.legend(['return rate = 78.74%'], loc='upper left', fontsize=20)
pyplot.title("Win return", fontsize=20)
pyplot.xlabel('trail', fontsize=20)
pyplot.ylabel('amount of bet', fontsize=20)
pyplot.show()
pyplot.savefig('winodds1.png')  

back testing for place

In [19]:
con1 = y_true<4
con2 = abs(np.array(y_true) - preds) <=3
con3 = y_true>0
preds_place = odd_df
preds_place_3 = preds_place[preds <=3]
place_return_df = preds_place_3['new_place odd']
place_return_df = place_return_df.fillna(-1)
place_return_df = np.array(place_return_df)
cum_place = np.cumsum(pd.DataFrame(place_return_df))*10
#cumsum plot
cum_place.plot()

pyplot.legend()
pyplot.title("Place return", fontsize=20)
pyplot.xlabel('trail', fontsize=20)
pyplot.ylabel('amount of bet', fontsize=20)
plt.legend(['return rate = 31.34%'], loc='upper left', fontsize=20)
pyplot.show()
pyplot.savefig('place1.png')  

plot xg boost

In [20]:
from numpy import loadtxt
from xgboost import XGBClassifier
from xgboost import plot_tree
import matplotlib.pyplot as plt
import graphviz
# load data
# fit model no training data
import os
os.environ["PATH"] += os.pathsep + r'C:\Users\admin\anaconda3\pkgs\graphviz-2.38-hfd603c8_2\Library\bin\graphviz'
# plot single tree
fig, ax = plt.subplots(figsize=(60, 60))
xgb.plot_tree(xg_clf,num_trees=2, ax=ax)
fig.set_size_inches(200, 200)
plt.show()
plt.savefig('xgb.png')